# Pip install ipywidgets to environment when running for first time

In [2]:
!pip install ipywidgets==8.1.1
!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


# Importing modules and setting up classes

In [3]:
from IPython.display import clear_output
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from datetime import datetime
import ipywidgets as widgets
import pandas as pd
import pandas as pd
import numpy as np
import requests
import os

In [4]:
# Setting up timing mappings to translate MATLAB syntax
TIMING_MAPPING = {
    "hourly": "H",
    "daily": "D",
    "minutely": "T"
}

REVERSE_TIMING_MAPPING = {v: k for k, v in TIMING_MAPPING.items()}

class CTDataProcessor:
    
    def __init__(self, timing="hourly"):
        self.timing = TIMING_MAPPING.get(timing, timing)

    def get_human_readable_timing(self):
        return REVERSE_TIMING_MAPPING.get(self.timing, self.timing)

    def process(self, filename="Results.csv"):

        # Read CT Data
        df = self.read_CT_data(filename)
        
        # get the list of deviceName
        device_names = self.list_machines(df)
        
        # Remove Power Failure detected rows
        df = df[df['powerFailureDetected'] != 1]

        # Export data for each machine
        for device in device_names:
            # Get current data
            temp_df = df[df['deviceName'] == device][["A", "channel1", "channel2", "channel3"]].dropna()
            
            # Aggregate data
            temp_df = temp_df.resample(self.timing).mean().fillna(0)

            # Placeholder columns
            temp_df["V"] = 0
            temp_df["kW"] = 0
            temp_df["cost"] = 0
            temp_df["neutral"] = 0
            
            directory_name = str(device)
            if not os.path.exists(directory_name):
                os.makedirs(directory_name)
                
            file_name = os.path.join(directory_name, f"{self.get_human_readable_timing()}_{device}.csv")
            temp_df.to_csv(file_name)

    def read_CT_data(self, filename):
        df = pd.read_csv(filename)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df.set_index('timestamp', inplace=True)
        print(df.head())
        return df.dropna()

    def list_machines(self, df):
        devices = df['deviceName'].unique()
        print(devices)
        return devices
    
class CTDataAnalyser:
    
    def __init__(self, MACHINE_NAMES, TIMING, PHASE, VOLTAGE, DAY_UNIT_COST, NIGHT_UNIT_COST, NIGHT_TARIFF_START_TIME, NIGHT_TARIFF_END_TIME, REGION):
        self.MACHINE_NAMES = MACHINE_NAMES
        self.TIMING = TIMING
        self.PHASE = PHASE
        self.VOLTAGE = VOLTAGE
        self.DAY_UNIT_COST = DAY_UNIT_COST
        self.NIGHT_UNIT_COST = NIGHT_UNIT_COST
        self.NIGHT_TARIFF_START_TIME = NIGHT_TARIFF_START_TIME
        self.NIGHT_TARIFF_END_TIME = NIGHT_TARIFF_END_TIME
        self.REGION = REGION

    def machine_calculations(self):
        
        # Getting adjusted timescale
        self.tscale = self.adjust_timescale()

        for device_name in self.MACHINE_NAMES:
            
            # Loading dataframe of individual machine
            device_df = pd.read_csv(f'{device_name}/{self.TIMING}_{device_name}.csv')

            device_df = self.compute_kW_utilization_and_cost(device_df, device_name)

            device_df = self.estimate_load_imbalance(device_df, device_name)

            # Carbon emission kgCO2/kWh
            # Currently skips this for minutely timings
            # if self.TIMING.lower() != 'minutely':
            #     if self.REGION.lower() == 'national':
            device_df = self.estimate_carbon_emissions(device_df)

            device_df.to_csv(f'{device_name}/{self.TIMING}_{device_name}.csv', index=False)

    # Calculates deviation from average between current channels for each individual current channel
    def unbalanced(self, channels):

        # Convert to numpy array for easier calculations
        channels = np.array(channels).T # Transpose to get channels as columns

        Iav = np.mean(channels, axis=1)

        u = np.max(np.abs(channels - Iav[:, np.newaxis]), axis=1) / Iav * 100
        u[np.isnan(u)] = 0
        
        return u
    
    def adjust_timescale(self):
        if self.TIMING == 'hourly':
            tscale = 1
        elif self.TIMING == 'minutely':
            tscale = 60
        elif self.TIMING == 'daily':
            tscale = 1/24
        elif self.TIMING == 'weekly':
            tscale = 1/(24*7)
        else:
            raise ValueError("Invalid timing value")
        return tscale
    
    def compute_kW_utilization_and_cost(self, device_df, device_name):
        # Only 3-phase Machines' Current should be multiplied by sqrt(3)
        device_df['A'] = device_df['A'] * (3 ** 0.5 if self.PHASE[self.MACHINE_NAMES.index(device_name)] == 3 else 1) / self.tscale

        # Insert the voltages of individual machines
        device_df['V'] = self.VOLTAGE[self.MACHINE_NAMES.index(device_name)]

        # Estimate kW
        device_df['kW'] = device_df['V'] * device_df['A'] / 1000

        # Estimate Utilization
        device_df['utilization'] = (device_df['A'] > 1).astype(int)

        # Estimate unit cost of electricity
        device_df['p'] = self.DAY_UNIT_COST[self.MACHINE_NAMES.index(device_name)]
        device_df['timestamp'] = pd.to_datetime(device_df['timestamp'])
        device_df.loc[device_df['timestamp'].dt.hour >= self.NIGHT_TARIFF_START_TIME, 'p'] = self.NIGHT_UNIT_COST[self.MACHINE_NAMES.index(device_name)]
        device_df.loc[device_df['timestamp'].dt.hour < self.NIGHT_TARIFF_END_TIME, 'p'] = self.NIGHT_UNIT_COST[self.MACHINE_NAMES.index(device_name)]
        
        device_df['cost'] = device_df['kW'] * device_df['p']
        return device_df
    
    def estimate_load_imbalance(self, device_df, device_name):
        if self.PHASE[self.MACHINE_NAMES.index(device_name)] == 1:
            device_df['unbalanced'] = self.unbalanced([device_df['channel1']])
        elif self.PHASE[self.MACHINE_NAMES.index(device_name)] == 3:
            device_df['unbalanced'] = self.unbalanced([device_df['channel1'], device_df['channel2'], device_df['channel3']])
        return device_df
    
    def estimate_carbon_emissions(self, device_df):
        st = str(device_df['timestamp'].iloc[0])
        en = str(device_df['timestamp'].iloc[-1])

        timestamp, co2 = self.fetch_carbon_emission_data(st, en)
        
        # Creating a DataFrame from the retrieved data
        tco2 = pd.DataFrame({'timestamp': timestamp, 'co2': co2})
        tco2['timestamp'] = pd.to_datetime(tco2['timestamp'])

        # Re-sampling operations
        # The MATLAB code was using 'retime' to handle the time series data.
        # `resample` and `interpolate` achieve a similar result.
        tco2.set_index('timestamp', inplace=True)
        tco2 = tco2.resample(TIMING_MAPPING.get(self.TIMING, self.TIMING)).mean().interpolate().reset_index()
        
        # Merging with the original DataFrame M
        device_df = pd.merge(device_df, tco2, on='timestamp', how='left')
        device_df['co2'].fillna(0, inplace=True)
        device_df['co2'] = device_df['kW'] * device_df['co2']
        return device_df

    def fetch_carbon_emission_data(self, st, en):
        # Convert to ISO8601 format and then replace the last characters to fit the required 'Z' format.
        st_iso = datetime.fromisoformat(st).isoformat().replace('+00:00', 'Z')
        en_iso = datetime.fromisoformat(en).isoformat().replace('+00:00', 'Z')

        url = f'https://api.carbonintensity.org.uk/intensity/{st_iso}/{en_iso}/'

         # Make a HTTP request to the URL with retries
        for _ in range(5):
            data = requests.get(url, timeout=10)
            # check if the response was successful 
            if(data.status_code == 200):
                data = data.json()
            else:
                print("HTTP request failed. Response code: " + str(data.status_code))

        # Extract useful data from the response
        timestamp = [datetime.fromisoformat(item['from'].replace('Z', '+00:00')).strftime('%Y-%m-%d %H:%M') for item in data['data']]
        co2 = [item['intensity']['actual'] / 1000 / self.tscale for item in data['data']]
        return timestamp, co2


# Run cell below to check what machines can be analysed

In [5]:
# Read the CSV file into a DataFrame
df = pd.read_csv("Results.csv")

# Get unique values from the 'Device Name' column
unique_names = df['deviceName'].unique()

print(', '.join([f'"{name}"' for name in unique_names]))

"Chiller", "BlowMoulder", "Desran"


# Define parameters for analysis

In [6]:
# Define timing: minutely, hourly or daily
TIMING = 'hourly'

# Machines
MACHINE_NAMES = ["Chiller", "BlowMoulder", "Desran"]

# Voltage
VOLTAGE = [240, 240, 240]

# Phase
PHASE = [3, 3, 3]

# Unit cost
DAY_UNIT_COST = [0.45, 0.45, 0.45]
NIGHT_UNIT_COST = [0.09, 0.09, 0.09]

# Night Tariff times - Ignore if both night & day tariffs are the same
# Assumed 10pm - 8am as night rate
NIGHT_TARIFF_START_TIME = 22
NIGHT_TARIFF_END_TIME = 8

# CO2 estimation region
REGION = 'national'

In [7]:
def validate_list_lengths(*lists):
    reference_length = len(lists[0])
    for lst in lists[1:]:
        if len(lst) != reference_length:
            raise ValueError("All lists must have the same length!")
            
validate_list_lengths(MACHINE_NAMES, VOLTAGE, PHASE, DAY_UNIT_COST, NIGHT_UNIT_COST)

# Run cell below to produce csvs with analysed data

In [8]:
processor = CTDataProcessor(TIMING)
processor.process()


to_remove = []  # List to keep track of devices with insufficient data

for device_name in MACHINE_NAMES[:]:  # Create a shallow copy for iteration
    device_df = pd.read_csv(f'{device_name}/{TIMING}_{device_name}.csv')

    # Skipping machines with insufficient data for analysis
    if len(device_df) < 2:
        print(f"Skipping {device_name} as only one line of data.\n")
        to_remove.append(device_name)

# Update MACHINE_NAMES to exclude devices with insufficient data
MACHINE_NAMES = [device for device in MACHINE_NAMES if device not in to_remove]

analyser = CTDataAnalyser(MACHINE_NAMES, TIMING, PHASE, VOLTAGE, DAY_UNIT_COST, NIGHT_UNIT_COST, NIGHT_TARIFF_START_TIME, NIGHT_TARIFF_END_TIME, REGION)
analyser.machine_calculations()

                     deviceId   deviceName           deviceType  \
timestamp                                                         
2023-09-27 17:09:58  0524A809      Chiller  3-CH-Current-Sensor   
2023-09-27 17:09:54  052043C3  BlowMoulder  3-CH-Current-Sensor   
2023-09-27 17:09:28  0524A809      Chiller  3-CH-Current-Sensor   
2023-09-27 17:09:24  052043C3  BlowMoulder  3-CH-Current-Sensor   
2023-09-27 17:08:57  0524A809      Chiller  3-CH-Current-Sensor   

                          location gatewayName  dBm      A  channel1  \
timestamp                                                              
2023-09-27 17:09:58  OaklandsGroup     wmgsme8  -75  16.33      16.3   
2023-09-27 17:09:54  OaklandsGroup     wmgsme8  -75  93.00      94.0   
2023-09-27 17:09:28  OaklandsGroup     wmgsme8  -76  16.30      16.2   
2023-09-27 17:09:24  OaklandsGroup     wmgsme8  -75  96.33      98.0   
2023-09-27 17:08:57  OaklandsGroup     wmgsme8  -75  16.30      16.3   

                     chan

/tmp/ipykernel_54174/4234536177.py:105: RuntimeWarning: invalid value encountered in divide
  u = np.max(np.abs(channels - Iav[:, np.newaxis]), axis=1) / Iav * 100


# Plot figures
Simply run code below to generate figures then use the interface below to interact with them

In [9]:
# Create a dictionary to store the generated figures for each device and type
figures = {}

for device_name in MACHINE_NAMES:
    device_df = pd.read_csv(f'{device_name}/{TIMING}_{device_name}.csv')
    
    # Create a function to generate and return a figure
    def create_figure(x, y, ylabel, title, color, legend_name=None):
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name=legend_name, line=dict(color=color, width=2)))
        fig.update_layout(title=title, xaxis_title='Timestamp', yaxis_title=ylabel, template='plotly_white')
        return fig

    # Generate figures using the function and store them in the dictionary
    figures[(device_name, 'CO_2')] = create_figure(device_df['timestamp'], device_df['co2'], 'kgCO_2', f'{device_name} - CO_2 Emission Profile', '#800000')
    figures[(device_name, 'kW')] = create_figure(device_df['timestamp'], device_df['kW'], 'kW', f'{device_name} - Power Consumption', 'black')
    figures[(device_name, 'Cost')] = create_figure(device_df['timestamp'], device_df['cost'], '£', f'{device_name} - Cost', 'black')
    figures[(device_name, 'Unbalanced')] = create_figure(device_df['timestamp'], device_df['unbalanced'], '%', f'{device_name} - Unbalanced Load', 'black')
    figures[(device_name, 'Unit Cost')] = create_figure(device_df['timestamp'], device_df['p'], '£ per kW', f'{device_name} - Unit Cost', 'black')

    # Separately generating figures for channels combining them into one figure
    fig_channels = go.Figure()
    for channel, color, name in zip(['channel1', 'channel2', 'channel3'], ['red', 'green', 'blue'], ['Channel 1', 'Channel 2', 'Channel 3']):

        fig_channels.add_trace(go.Scatter(x=device_df['timestamp'], y=device_df[channel], mode='lines', name=name, line=dict(color=color, width=2)))
    fig_channels.update_layout(title=f'{device_name} - Channels', xaxis_title='Timestamp', yaxis_title='Channels', template='plotly_white')
    figures[(device_name, 'Channels')] = fig_channels

# Create dropdowns for device names and graph types
device_dropdown = widgets.Dropdown(options=MACHINE_NAMES, description='Device:')
graph_type_dropdown = widgets.Dropdown(options=['CO_2', 'kW', 'Channels', 'Cost', 'Unbalanced', 'Unit Cost'], description='Graph Type:')

out = widgets.Output()  # Create output widget


# Display the dropdowns and the initial graph
def update_graph(device, graph_type):
    with out:
        clear_output(wait=True)
        # Checking if selection exists in dictionary and displaying accordingly
        if (device, graph_type) in figures:
            figures[(device, graph_type)].show()

# Display the output widget (where the graph will be shown)
display(out)

# Link the update_graph function to the dropdowns
widgets.interactive(update_graph, device=device_dropdown, graph_type=graph_type_dropdown)


Output()

interactive(children=(Dropdown(description='Device:', options=('Chiller', 'BlowMoulder'), value='Chiller'), Dr…

# Calculate total kW between datetime
Modify the datetimes below and then run code to calculate results

In [10]:
from_datetime = pd.Timestamp('2023-09-18 00:00:00')
to_datetime = pd.Timestamp('2023-09-19 00:00:00')

In [11]:
for device_name in MACHINE_NAMES:
    device_df = pd.read_csv(f'{device_name}/{TIMING}_{device_name}.csv')
    device_df['timestamp'] = pd.to_datetime(device_df['timestamp'])

    # Extracting data within the datetime range
    device_data_range = device_df[(device_df['timestamp'] >= from_datetime) & (device_df['timestamp'] <= to_datetime)]

    print("\n----------------------{}--------------------".format(device_name))
    
    total_kW = device_data_range['kW'].sum()
    print("\ntotal kW between {} and {} is = {:.2f} kW".format(from_datetime, to_datetime, total_kW))
    
    if TIMING.lower() != "minutely":
        total_co2 = device_data_range['co2'].sum()
        print("\ntotal CO2 emissions between {} and {} is = {:.2f} kg".format(from_datetime, to_datetime, total_co2))
    else:
        print('\nCannot estimate CO2 if the timing = "minutely"')

    total_cost = device_data_range['cost'].sum()
    print("\ntotal cost between {} and {} is = £{:.2f}".format(from_datetime, to_datetime, total_cost))
    
    avg_channel1 = device_data_range['channel1'][device_data_range['channel1'] != 0].mean()
    avg_channel2 = device_data_range['channel2'][device_data_range['channel2'] != 0].mean()
    avg_channel3 = device_data_range['channel3'][device_data_range['channel3'] != 0].mean()
    print("\nAverage (non-zero) Current (A) in each channels between {} and {} are = [{:.2f} A, {:.2f} A, {:.2f} A]".format(from_datetime, to_datetime, avg_channel1, avg_channel2, avg_channel3))
    
    total_unbalanced = device_data_range['unbalanced'][device_data_range['unbalanced'] != 0].mean()
    print("\nAverage (non-zero) load imbalance between {} and {} is = {:.2f}%".format(from_datetime, to_datetime, total_unbalanced))


----------------------Chiller--------------------

total kW between 2023-09-18 00:00:00 and 2023-09-19 00:00:00 is = 172.81 kW

total CO2 emissions between 2023-09-18 00:00:00 and 2023-09-19 00:00:00 is = 8.70 kg

total cost between 2023-09-18 00:00:00 and 2023-09-19 00:00:00 is = £65.56

Average (non-zero) Current (A) in each channels between 2023-09-18 00:00:00 and 2023-09-19 00:00:00 are = [27.96 A, 28.33 A, 26.85 A]

Average (non-zero) load imbalance between 2023-09-18 00:00:00 and 2023-09-19 00:00:00 is = 3.12%

----------------------BlowMoulder--------------------

total kW between 2023-09-18 00:00:00 and 2023-09-19 00:00:00 is = 579.43 kW

total CO2 emissions between 2023-09-18 00:00:00 and 2023-09-19 00:00:00 is = 29.17 kg

total cost between 2023-09-18 00:00:00 and 2023-09-19 00:00:00 is = £218.67

Average (non-zero) Current (A) in each channels between 2023-09-18 00:00:00 and 2023-09-19 00:00:00 are = [95.94 A, 98.99 A, 83.84 A]

Average (non-zero) load imbalance between 202